# Utilisation des outils
Dans cet exercice, nous allons franchir une étape clé dans la construction d’un agent intelligent : permettre au modèle de langage d’interagir avec des outils externes. Jusqu’à présent, le LLM se contentait de raisonner et de produire du texte. Or, pour répondre à des questions ancrées dans le monde réel (date actuelle, météo, données externes…), il est nécessaire de lui donner accès à des fonctions capables d’effectuer des actions concrètes.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [ ]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')

## Configuration du llm sur dspy

In [ ]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

<ins>**Exercice:**</ins> construire un agent capable de répondre à une question utilisateur du type : « Quelle est la température actuelle pour la ville de Douala ? » 

## Création des outils
Pour créer une fonction utilisable comme outil dans DSPy, il suffit de définir une fonction Python classique avec une signature claire et des paramètres typés, puis de lui associer une docstring explicite décrivant précisément son rôle. DSPy s’appuie sur cette signature et sur la documentation pour comprendre quand et comment appeler l’outil au cours du raisonnement de l’agent.

Il vous faudra donc déterminer les outils nécessaires à créer pour répondre aux besoins.

<details>
    <summary>Voir plus d'indices</summary>
    Trois fonctions sont nécessaires:
    <ul>
        <li><code>get_current_datetime</code>: Cette fonction retourne la date au format iso et permettra au llm d'avoir l'information de la date et heure courante au moment de l'exécution, pour ce faire vous pouvez utiliser la librarie `datetime` de python.</li>
        <li><code>get_geolocalisation</code> : Cette fonction prend en entrée le nom d’une ville (<code>town_name</code>) et permet au LLM d'obtenir sa position géographique. Elle interroge l'API de géocodage d'Open-Meteo (<code>f'https://geocoding-api.open-meteo.com/v1/search/v1/search</code>) avec les paramètres <code>name</code> et <code>count=1</code>, puis retourne la latitude et la longitude du premier résultat.
        </li>
        <li><code>get_temperature_forecast</code> : Cette fonction utilise les coordonnées géographiques (<code>latitude</code> et <code>longitude</code>) pour récupérer les prévisions de température. Elle appelle l'API de prévisions météo d’Open-Meteo (<code>f'https://api.open-meteo.com/v1/forecast/v1/forecast</code>) en précisant les paramètres <code>hourly=temperature_2m</code>, <code>forecast_days=1</code> et <code>format=json</code>. La fonction retourne une liste structurée associant chaque date ou heure à la température correspondante, afin que le LLM puisse facilement exploiter ces données dans sa réponse finale.
        </li>
    </ul>
</details>

In [ ]:
import requests
from datetime import datetime
                         
class ForecastTools():
    """Forecast set of tools"""

    

## Créer une prédiction basée sur les outils
Quel module DSPy permet de créer ce type de traitement ? Trouvez la réponse à cette question et adaptez le code suivant

In [ ]:
tools = ForecastTools()
assistant = dspy.Predict("question -> answer")

## Exécution

In [ ]:
from rich import print

response = assistant(question="Quelle est la température actuelle pour la ville de Douala ?")

print(response)
print(lm.history)
dspy.inspect_history()